In [1]:
import h5py as h5
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

train_folder = 'D:\\Datasets\\g2net-detecting-continuous-gravitational-waves (1)\\train'
test_folder = 'D:\\Datasets\\g2net-detecting-continuous-gravitational-waves (1)\\test'

# List of names of all files in the folder
train_files = os.listdir(train_folder)
test_files = os.listdir(test_folder)

In [2]:
# Get train_labels and associate them with appropriate files

train_labels = 'D:\\Datasets\\g2net-detecting-continuous-gravitational-waves (1)\\train_labels.csv'

# Create a dictionary with train_files values as keys and train_labels values as values
train_labels_dict = {}
with open(train_labels, 'r') as f:
    for line in f:
        (key, val) = line.split(',')
        train_labels_dict[key] = val

# Remove "\n" from the end of the values
for key in train_labels_dict:
    train_labels_dict[key] = train_labels_dict[key].rstrip()

In [3]:
# Get contents of first train file

with h5.File(train_folder + '\\' + train_files[1], 'r') as f:
    for file_key in f.keys():
        group = f[file_key]
        print(group)
        try:
            for group_key in group.keys():
                group2 = group[group_key]
                print(f"---->{group2}")
                for group_key2 in group2.keys():
                    print(f"--------->{group2[group_key2]}")
        except AttributeError:
            pass


<HDF5 group "/004f23b2d" (3 members)>
----><HDF5 group "/004f23b2d/H1" (2 members)>
---------><HDF5 dataset "SFTs": shape (360, 4546), type "<c8">
---------><HDF5 dataset "timestamps_GPS": shape (4546,), type "<i8">
----><HDF5 group "/004f23b2d/L1" (2 members)>
---------><HDF5 dataset "SFTs": shape (360, 4630), type "<c8">
---------><HDF5 dataset "timestamps_GPS": shape (4630,), type "<i8">
----><HDF5 dataset "frequency_Hz": shape (360,), type "<f8">


In [4]:
def traverse_hdf5(hdf5_file, file_name):
    data = []
    # Open the HDF5 file
    with h5.File(hdf5_file, 'r') as f:
        # Recursively traverse through the groups in the HDF5 file
        def traverse(name, path):
            # Get the object at the current path
            obj = f[name]
            # If the object is a group, traverse through it
            if isinstance(obj, h5.Group):
                for key in obj.keys():
                    traverse(f"{name}/{key}", f"{path}/{key}")
            # If the object is a dataset, store the data and the dataset name in the list
            elif isinstance(obj, h5.Dataset):
                data.append((obj[()], path, file_name))
        traverse("/", "/")
    # Create a Pandas dataframe from the data
    df = pd.DataFrame(data, columns=["data", "name", "file"])

    # Rename the "Filename" column to "id"
    df.rename(columns={'file': 'Id'}, inplace=True)

    return df

In [5]:
# Get contents of first train file
df = traverse_hdf5(train_folder + '\\' + train_files[0], train_files[0])

# Get length of arrays within the data column
df['data_length'] = df['data'].apply(lambda x: len(x))

# Get first list of first cell in data column
df

,data,name,Id,data_length
0,"[[(-2.017801e-24+1.7066067e-22j), (-1.4645843e...",//001121a05/H1/SFTs,001121a05.hdf5,360
1,"[1238170021, 1238171821, 1238173621, 123817542...",//001121a05/H1/timestamps_GPS,001121a05.hdf5,4612
2,"[[(1.643243e-22-3.1472818e-23j), (-1.0013719e-...",//001121a05/L1/SFTs,001121a05.hdf5,360
3,"[1238169614, 1238171414, 1238173214, 123817501...",//001121a05/L1/timestamps_GPS,001121a05.hdf5,4653
4,"[53.50777777777778, 53.50833333333334, 53.5088...",//001121a05/frequency_Hz,001121a05.hdf5,360


In [11]:
# Create df, with columns representing timestamps from second data cell, and rows representing the first data cell
df2 = pd.DataFrame(df['data'][0])

# Column names are the timestamps
df2.columns = df['data'][1]

# Add average of column values as a new row
df2.loc['average'] = df2.mean()

# Add standard deviation of column values as a new row
df2.loc['std'] = df2.std()

# Add skewness of column values as a new row
df2.loc['skew'] = df2.skew()

# Add kurtosis of column values as a new row
df2.loc['kurtosis'] = df2.kurtosis()

df2

C:\Users\noudy\DataSpellProjects\Gravitational\DS_venv\lib\site-packages\numpy\core\_methods.py:48: ComplexWarning: Casting complex values to real discards the imaginary part
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
C:\Users\noudy\DataSpellProjects\Gravitational\DS_venv\lib\site-packages\pandas\core\nanops.py:873: RuntimeWarning: invalid value encountered in sqrt
  result = np.sqrt(nanvar(values, axis=axis, skipna=skipna, ddof=ddof, mask=mask))
C:\Users\noudy\DataSpellProjects\Gravitational\DS_venv\lib\site-packages\pandas\core\nanops.py:1163: ComplexWarning: Casting complex values to real discards the imaginary part
  values = values.astype("f8")
C:\Users\noudy\DataSpellProjects\Gravitational\DS_venv\lib\site-packages\pandas\core\nanops.py:1250: ComplexWarning: Casting complex values to real discards the imaginary part
  values = values.astype("f8")


,1238170021,1238171821,1238173621,1238175421,1238177221,1238179021,1238180821,1238182621,1238184421,1238186221,...,1248539639,1248541439,1248543239,1248545039,1248546839,1248548639,1248550439,1248552239,1248554039,1248555839
0,-2.017801e-24+1.706607e-22j,-1.464584e-23+9.642138e-23j,1.391039e-23-4.392366e-23j,2.934738e-23-1.632859e-22j,-7.205563e-23-1.428877e-22j,1.509542e-22-7.776660e-23j,1.195127e-22+1.074785e-22j,-5.966815e-23+3.680174e-23j,2.665954e-24+8.128846e-23j,1.612900e-22+5.822668e-23j,...,3.082725e-22-3.613544e-23j,1.205249e-22+9.118960e-23j,2.861275e-23+7.403217e-23j,-2.749051e-23-1.968903e-22j,-3.948799e-23-2.754519e-23j,-1.176302e-22-1.099499e-22j,-3.110633e-23+7.841416e-23j,-1.717718e-23+1.283729e-22j,1.389211e-22-8.297767e-23j,7.272958e-23+4.455958e-23j
1,-1.871139e-22+1.007350e-22j,-1.510940e-22+9.367457e-24j,5.019298e-23-5.209825e-23j,-1.086745e-22+1.983568e-23j,2.467436e-23+2.779556e-23j,9.644749e-23+6.473138e-24j,2.512335e-22-4.792071e-23j,-1.068056e-22+7.975918e-23j,2.103226e-22+1.184502e-22j,-4.295231e-23-1.334193e-22j,...,-9.865622e-23+9.955658e-24j,-1.077493e-22-3.276680e-23j,4.444153e-23-1.241720e-23j,2.974329e-23+4.581187e-23j,-2.172054e-22-3.942463e-23j,4.186732e-23+4.143750e-23j,-2.277401e-23-1.501246e-22j,-5.972212e-23-1.090459e-22j,8.073840e-23+1.614888e-22j,1.476014e-22+1.079513e-22j
2,-1.901606e-22+1.526898e-22j,1.059649e-22+6.504072e-23j,1.296340e-22+5.387476e-23j,-3.108431e-23+1.989968e-22j,2.818042e-23-8.323839e-23j,-1.607108e-23-1.110477e-23j,-1.335912e-22+1.271735e-22j,-1.039643e-22-1.190458e-22j,-3.244993e-22-4.978333e-23j,-5.256758e-23-3.404730e-23j,...,-7.674154e-23+1.223068e-22j,-1.358693e-22+3.920475e-23j,-1.100504e-23+1.978281e-22j,-4.468216e-24+4.729299e-23j,-1.707596e-23-1.994347e-22j,6.213242e-23+3.120494e-24j,1.203371e-22-2.482423e-23j,-1.532590e-22-5.962312e-23j,-6.094774e-23+2.810304e-23j,-3.277597e-23-6.076202e-23j
3,9.034575e-24-9.360981e-23j,5.172909e-23-2.954975e-22j,1.097326e-23-8.117022e-23j,7.346526e-23-1.533208e-22j,8.429252e-23-6.141384e-23j,-1.269745e-22+1.190623e-23j,-3.062227e-23-4.818979e-23j,-1.871258e-22-6.194506e-23j,-4.915285e-23+2.990681e-24j,-8.661373e-24+6.903788e-23j,...,-6.427304e-23-9.979299e-23j,1.491164e-23+4.381242e-23j,2.800941e-23-1.055062e-22j,-9.645621e-23-9.717640e-23j,-1.583596e-22+2.798871e-23j,-1.874950e-22+1.136141e-22j,-2.243608e-22+1.776946e-23j,-1.161252e-22+9.824507e-23j,1.351628e-22+1.285419e-22j,-1.860812e-22-1.007592e-23j
4,3.884536e-23+9.743752e-23j,6.932494e-23+3.436815e-23j,4.751554e-23+3.152470e-24j,-7.239878e-23-1.983319e-23j,1.146661e-22+5.217264e-23j,6.342020e-23-1.503384e-22j,4.285598e-23+1.049940e-22j,-6.285604e-23-1.200224e-22j,4.498359e-23-1.720343e-23j,9.865346e-24-1.102503e-22j,...,-1.720122e-22-1.277606e-22j,9.099982e-23+1.089591e-22j,1.129112e-22+8.876515e-23j,3.823606e-23-1.955166e-22j,-3.148861e-22-4.656572e-23j,-2.145164e-22+1.032763e-22j,1.221336e-22-7.266460e-23j,-2.423047e-22-2.126750e-22j,1.120399e-22-7.913659e-23j,-2.396455e-23+5.139624e-23j
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,-1.407118e-23-1.133035e-22j,2.308325e-23-5.763252e-23j,-2.879641e-23-2.569706e-23j,3.730476e-23-7.881608e-23j,3.248028e-23+7.633730e-23j,-8.439254e-23+8.439179e-23j,-1.340824e-22-1.744120e-23j,1.030267e-22-9.324072e-24j,-5.184786e-23-8.024168e-23j,-1.247131e-23-1.352136e-22j,...,-1.806964e-22+3.261813e-23j,8.000130e-24+1.289507e-22j,1.561291e-22+1.160848e-22j,3.044505e-23+1.527895e-22j,1.793383e-23+1.616275e-22j,-1.846653e-23-1.253062e-22j,2.145384e-23-1.410290e-22j,-5.617747e-23+5.003053e-24j,5.271244e-23-9.958076e-24j,-6.401009e-23+6.848476e-23j
average,-3.242841e-24+2.765646e-24j,8.342572e-24+7.603134e-25j,-5.016152e-24+7.961558e-24j,5.215874e-24-7.018373e-25j,-5.592722e-24+9.246273e-24j,4.394467e-24+3.809551e-25j,6.506052e-24+8.045235e-24j,6.217769e-24+9.650681e-25j,4.318668e-25+8.084194e-24j,3.543292e-25+4.572641e-24j,...,8.692746e-24-4.267887e-24j,2.742198e-24+6.172858e-24j,-1.4319

In [ ]:
# Show a trend line for the average row


In [19]:
# Function to convert scientific notation to float
def convert_to_float(x):
    return float(x)

# Convert the values in the "Avg H1-SFTs" column to float
complete_df_train['Avg H1-SFTs'] = complete_df_train['Avg H1-SFTs'].apply(convert_to_float)

complete_df_train

C:\Users\noudy\DataSpellProjects\Gravitational\DS_venv\lib\site-packages\ipykernel_launcher.py:3: ComplexWarning: Casting complex values to real discards the imaginary part
  This is separate from the ipykernel package so we can avoid doing imports until


,Id,Avg H1-SFTs,Avg L1-SFTs,Avg Frequency_Hz
0,001121a05.hdf5,-2.787475e-26,(-7.563195e-26+1.6791384e-26j),53.6075
0,004f23b2d.hdf5,5.795783e-26,(-6.577777e-26-9.961035e-27j),329.119722
0,00a6db666.hdf5,1.779411e-25,(3.6714905e-26+1.6741528e-26j),213.288056
0,00f36a6ac.hdf5,2.703786e-26,(-9.8693624e-26-3.6599078e-26j),453.359167
0,010a387db.hdf5,-7.878897e-26,(6.442983e-26-2.6589115e-26j),231.983611
...,...,...,...,...
0,fe38dbe64.hdf5,1.986287e-27,(2.9517718e-26-4.4535038e-26j),373.019722
0,feafd0d16.hdf5,-3.973601e-26,(5.1489706e-26-6.1739946e-26j),481.319722
0,feeca844e.hdf5,-3.382550e-26,(4.5678313e-27-6.1452293e-28j),369.324722
0,ff5ad023f.hdf5,5.578002e-26,(-6.8264504e-26+4.6519235e-26j),293.868056


In [20]:
# Convert the values in the "Avg L1-SFTs" column to float
complete_df_train['Avg L1-SFTs'] = complete_df_train['Avg L1-SFTs'].apply(convert_to_float)

complete_df_train

C:\Users\noudy\DataSpellProjects\Gravitational\DS_venv\lib\site-packages\ipykernel_launcher.py:3: ComplexWarning: Casting complex values to real discards the imaginary part
  This is separate from the ipykernel package so we can avoid doing imports until


,Id,Avg H1-SFTs,Avg L1-SFTs,Avg Frequency_Hz
0,001121a05.hdf5,-2.787475e-26,-7.563195e-26,53.6075
0,004f23b2d.hdf5,5.795783e-26,-6.577777e-26,329.119722
0,00a6db666.hdf5,1.779411e-25,3.671491e-26,213.288056
0,00f36a6ac.hdf5,2.703786e-26,-9.869362e-26,453.359167
0,010a387db.hdf5,-7.878897e-26,6.442983e-26,231.983611
...,...,...,...,...
0,fe38dbe64.hdf5,1.986287e-27,2.951772e-26,373.019722
0,feafd0d16.hdf5,-3.973601e-26,5.148971e-26,481.319722
0,feeca844e.hdf5,-3.382550e-26,4.567831e-27,369.324722
0,ff5ad023f.hdf5,5.578002e-26,-6.826450e-26,293.868056


In [21]:
# Remove ".hdf5" from the end of the Id column for each row
complete_df_train['Id'] = complete_df_train['Id'].str[:-5]

labels_df = pd.read_csv(train_labels)

labels_df = labels_df.rename(columns={"id": "Id"})

complete_df_train = pd.merge(complete_df_train, labels_df, on="Id", how="inner")

In [22]:
complete_df_train

,Id,Avg H1-SFTs,Avg L1-SFTs,Avg Frequency_Hz,target
0,001121a05,-2.787475e-26,-7.563195e-26,53.6075,1
1,004f23b2d,5.795783e-26,-6.577777e-26,329.119722,1
2,00a6db666,1.779411e-25,3.671491e-26,213.288056,1
3,00f36a6ac,2.703786e-26,-9.869362e-26,453.359167,1
4,010a387db,-7.878897e-26,6.442983e-26,231.983611,1
...,...,...,...,...,...
598,fe38dbe64,1.986287e-27,2.951772e-26,373.019722,1
599,feafd0d16,-3.973601e-26,5.148971e-26,481.319722,1
600,feeca844e,-3.382550e-26,4.567831e-27,369.324722,1
601,ff5ad023f,5.578002e-26,-6.826450e-26,293.868056,1


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Separate the independent variables (predictor variables) from the dependent variable (response variable)
X = complete_df_train["Avg Frequency_Hz"].values.reshape(-1, 1)
y = complete_df_train["target"]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.24, random_state=42)

# Create an instance of the LogisticRegression model
model = KNeighborsClassifier(n_neighbors = 5,metric = 'minkowski',p = 2)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate the accuracy, precision, recall, and F1 score
accuracy = accuracy_score(y_test, y_pred)


print("Accuracy:", accuracy)
y_pred


Accuracy: 0.593103448275862


array([ 1,  0,  1,  1,  1, -1,  0,  1,  1,  1,  1,  1,  0,  0,  1,  1,  0,
        1,  0,  1,  1,  0,  1,  0,  1,  1,  1,  1,  1,  0,  0,  1,  1,  1,
        0,  1,  1,  0,  1,  1,  1,  1,  1,  0,  0,  0,  1,  1,  1,  1,  1,
        1,  1,  1,  0,  1,  1,  1,  0,  1,  1,  0,  1,  1,  1,  1,  0,  1,
        1,  1,  0,  1,  1,  0,  0,  0,  1,  0,  1,  0,  1,  1,  1,  0,  0,
        1,  0,  1,  1,  1,  0,  0,  1,  1,  0,  0,  1,  0,  0,  0,  1,  1,
        1,  1,  1,  0,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  0,
        1,  1,  0,  1,  1,  1,  1,  1,  1,  1,  0,  1,  0,  1,  1,  0,  1,
        1,  1,  0,  0,  0,  1,  1,  1,  1], dtype=int64)

In [25]:
complete_df_test = pd.DataFrame(columns=["Id", "Avg H1-SFTs", "Avg L1-SFTs", "Avg Frequency_Hz"])

# Execute the traverse_hdf5 function on all files in the train folder and replace complete_df with complete_df + df
for file in test_files:
    df = traverse_hdf5(test_folder + '\\' + file, file)
    complete_df_test = complete_df_test.append(df)
    
# Function to convert scientific notation to float
def convert_to_float(x):
    return float(x)


# Remove ".hdf5" from the end of the Id column for each row
complete_df_test['Id'] = complete_df_test['Id'].str[:-5]

# Convert the values in the "Avg H1-SFTs" column to float
complete_df_test['Avg H1-SFTs'] = complete_df_test['Avg H1-SFTs'].apply(convert_to_float)

# Convert the values in the "Avg L1-SFTs" column to float
complete_df_test['Avg L1-SFTs'] = complete_df_test['Avg L1-SFTs'].apply(convert_to_float)


print(complete_df_test.head())

          Id   Avg H1-SFTs   Avg L1-SFTs Avg Frequency_Hz
0  00054c878 -2.864107e-26  1.970546e-26       306.501944
0  0007285a3 -2.269763e-26  1.093008e-25       126.448611
0  00076c5a6 -3.871007e-26 -3.364705e-26       197.389167
0  001349290  6.066534e-26  6.196433e-26        95.008056
0  001a52e92  8.669040e-26  8.671533e-27       464.176944


C:\Users\noudy\DataSpellProjects\Gravitational\DS_venv\lib\site-packages\ipykernel_launcher.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
  # Remove the CWD from sys.path while we load stuff.


In [13]:
X_test = complete_df_test.drop(columns=["Id"])

# Make predictions on the test data
y_pred_test = model.predict(X_test)

y_pred_test

array([1, 1, 1, ..., 0, 1, 0], dtype=int64)

In [16]:
# Create csv file with first column being test_files without the .hdf5 extension and second column being the predictions
submission = pd.DataFrame({'Id': test_files, 'target': y_pred_test})
submission['Id'] = submission['Id'].str[:-5]
submission.to_csv('submission.csv', columns=["Id", "target"], index=False)

In [15]:
submission['target'].value_counts()

 1    5847
 0    2040
-1      88
Name: target, dtype: int64